In [1]:
import os
from PIL import Image, ImageFile
import math
import copy
import wandb
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
from torch.autograd import Variable
import torch.cuda as cuda
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from utils import *


/Users/pongkornsettasompop/anaconda3/envs/mi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def checkImage(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
    pass

img_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    ])

non_augmentned = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    ])

gray = transforms.Compose([
    transforms.Grayscale()
    ])

augmentations = {
    'rotation': transforms.Compose([transforms.RandomRotation(degrees=(0,90))]),
    'rotation2': transforms.Compose([transforms.RandomRotation(degrees=(270,360))]),
    'norm': transforms.Compose([transforms.Normalize(mean=[0.42, 0.42, 0.42],std=[0.42, 0.42, 0.42] )]),
    'affine': transforms.Compose([ transforms.RandomAffine(degrees=(0, 90), scale=(0.5, 0.9))]),
    'affine2': transforms.Compose([ transforms.RandomAffine(degrees=(270, 360), scale=(0.5, 0.9))]),
    'blur': transforms.Compose([transforms.GaussianBlur(kernel_size=(3, 7), sigma=(0.1, 3))]),
    'sharpness': transforms.Compose([transforms.RandomAdjustSharpness(40)]),
    'contrast': transforms.Compose([transforms.ColorJitter(contrast=(1,2))]),
}


def imshow(img):
  ''' function to show image '''
  #img = img / 2 + 0.5 # unnormalize
  npimg = img.numpy() # convert to numpy objects
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()


In [ ]:
base_dir = './dataset/'
print(os.listdir(base_dir))
for base in os.listdir(base_dir):
    if base == "train":
        train_datasets = []
        for augmentation_name, augmentation_transform in augmentations.items():
            combined_transform = transforms.Compose([img_transforms,augmentation_transform])
            dataset = torchvision.datasets.ImageFolder(
                root=os.path.join(base_dir, base),
                transform=combined_transform,  # Combine main and augmentation transforms
                is_valid_file=checkImage
            )
            train_datasets.append(dataset)

        new_train = torch.utils.data.ConcatDataset(train_datasets)
        
        
    elif base == "val":
        val_datasets = []
        for augmentation_name, augmentation_transform in augmentations.items():
            combined_transform = transforms.Compose([img_transforms,augmentation_transform])
            print(combined_transform)
            dataset = torchvision.datasets.ImageFolder(
                root=os.path.join(base_dir, base),
                transform=combined_transform,  # Combine main and augmentation transforms
                is_valid_file=checkImage
            )
            val_datasets.append(dataset)

        new_val = torch.utils.data.ConcatDataset(val_datasets)
    elif base == "test":
        test_data = torchvision.datasets.ImageFolder(root = os.path.join(base_dir,base), transform = non_augmentned, is_valid_file = checkImage)
        new_test = test_data


In [ ]:
#By default, PyTorch’s data loaders are set to a batch_size of 1.
BATCH_SIZE = 1000
train_data_loader = torch.utils.data.DataLoader(new_train, batch_size = BATCH_SIZE,shuffle=False, num_workers=2)
val_data_loader  = torch.utils.data.DataLoader(new_val, batch_size = BATCH_SIZE,shuffle=True, num_workers=2) 
test_data_loader  = torch.utils.data.DataLoader(new_test, batch_size = 2000,shuffle=False, num_workers=2)


num_step =math.ceil(len(train_data_loader.dataset) / BATCH_SIZE)

print(type(train_data_loader))


In [ ]:
# classes = ('scgc','scgp','wedo')

# sample = next(iter(train_data_loader))
# imgs, lbls = sample
# print(lbls)
# print(imgs.shape)

# # call function on our images # +29
# origi_img_pos = 16
# step = 59
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos ],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*2)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*3)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*4)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*5)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*6)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs[origi_img_pos +(step*7)],nrow=15,padding=20))
# imshow(torchvision.utils.make_grid(imgs,nrow=15,padding=20))
# print(' '.join('%s' % classes[lbls[j]] for j in range(len(imgs))))


In [ ]:
from model import *
from torchsummary import summary
net = ResNet(ResidualBlock, [3, 4, 6, 3],num_classes=3)
summary(net, (3, 224, 224))
print(net)

In [ ]:
 #wand setup
#"weightname":"S12_3-9_fir8-14"
#name=f"CNN_S12_3-9_fir8-14",
wandb.login()
wand = wandb.init(
      # Set the project where this run will be logged
      project="AR-classification", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"res50_AR_12", 
      # Track hyperparameters and run metadata
      # 0.0000001
      config={
      "learning_rate": 0.0001,
      "architecture": "CNN",
      "dataset": "Organization",
      "epochs": 200,
      "weightname":"res50_12",
      "num_step_per_epoch" : num_step
      }
    )

In [ ]:
config = wand.config
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=config.learning_rate, weight_decay = 0.001, momentum = 0.9)


In [ ]:
train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_data_loader,
    loader_test =test_data_loader,
    vail_loader = val_data_loader,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

In [ ]:
PATH = f"./save_weight/res50_12/0.3732_res50_12_0.3732_82.7586.pth"
# # reload
net = ResNet(ResidualBlock, [3, 4, 6, 3])
net.load_state_dict(torch.load(PATH))


In [ ]:
classes = ('scgc','scgp','wedo')
sample = next(iter(test_data_loader))
imgs, lbls = sample
print(type(imgs))
# print images
imshow(torchvision.utils.make_grid(imgs))
print('GroundTruth: ', ' '.join('%s' % classes[lbls[j]] for j in  range(len(imgs))))

outputs = net(imgs)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%s' % classes[predicted[j]]
                              for j in range(len(imgs))))

wrong_img = []
for i in range(len(imgs)):
    if classes[lbls[i]] != classes[predicted[i]]:
        wrong_img.append(imgs[i])


In [ ]:
print(len(wrong_img))
imshow(torchvision.utils.make_grid(wrong_img))

In [ ]:
from sklearn import metrics
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(labels)
        print(predicted)
        
        print(total)
        print(correct)

confusion_matrix = metrics.confusion_matrix(labels, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['scgc','scgp','wedo'])

cm_display.plot()
plt.show()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))